# 13. Parallelizing Neural Netowrk Training with TensorFlow

- 13.01. TensorFlow and training performance
  - 13.01.01. Performance challenges
  - 13.01.02. What is TensorFlow?
  - 13.01.03. How we will learn TensorFlow
- 13.02. First steps with TensorFlow
  - 13.02.01. Installing TensorFlow
  - 13.02.02. Creating tensors in TensorFlow
  - 13.02.03. Manipulating the data type and shape of a tensor
  - 13.02.04. Applying mathematical operations to tensors
  - 13.02.05. Split, stack, and concatenate tensors
  - 13.02.06. Building input pipelines using tf.data – the TensorFlow Dataset API
    - 13.02.06.01. Creating a TensorFlow Dataset from existing tensors
    - 13.02.06.02. Combining two tensors into a joint dataset
    - 13.02.06.03. Shuffle, batch, and repeat
    - 13.02.06.04. Creating a dataset from files on your local storage disk
    - 13.02.06.05. Fetching available datasets from the `tensorflow_datasets` library
- 13.03. Building an NN model in TensorFlow
  - 13.03.01. The TensorFlow Keras API (tf.keras)
  - 13.03.02. Building a linear regression model
  - 13.03.03. Model training via the `.compile()` and `.fit()` methods 
  - 13.03.04. Building a multilayer perceptron for classifying flowers in the Iris dataset
  - 13.03.05. Evaluating the trained model on the test dataset
  - 13.03.06. Saving and reloading the trained model
- 13.04. Choosing activation functions for multilayer NNs
  - 13.04.01. Logistic function recap
  - 13.04.02. Estimating class probabilities in multiclass classification via the softmax function
  - 13.04.03. Broadening the output spectrum using a hyperbolic tangent
  - 13.04.04. Rectified linear unit activation
- 13.05. Summary

In [1]:
%run -i 'watermark.py'

2020-09-12 10:48:06
----------------------
python		3.6.7
----------------------
numpy		1.18.5
scipy		1.4.1
pandas		0.25.1
matplotlib	3.1.1
imageio		2.5.0
----------------------
ipython		7.8.0
----------------------
sklearn		0.20.4
tensorflow	2.3.0
nltk		3.2.4
----------------------
networkx	2.0


## 13.01. TensorFlow and training performance

### 13.01.01. Performance challenges

### 13.01.02. What is TensorFlow?

![Fig.13.1](https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch13/images/01.png)

### 13.01.03. How we will learn TensorFlow

## 13.02. First steps with TensorFlow

### 13.02.01. Installing TensorFlow

### 13.02.02. Creating ttensors in TensorFlow



In [2]:
#import tensorflow as tf
#import numpy as np

np.set_printoptions(precision=3)
a = np.array([1, 2, 3], dtype=np.int32)
b = [4, 5, 6]
t_a = tf.convert_to_tensor(a)
t_b = tf.convert_to_tensor(b)
print(t_a)
print(t_b)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([4 5 6], shape=(3,), dtype=int32)


In [3]:
t_ones = tf.ones((2, 3))
t_ones.shape

TensorShape([2, 3])

In [4]:
t_ones.numpy()

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [5]:
const_tensor = tf.constant([1.2, 5, np.pi], dtype=tf.float32)
print(const_tensor)

tf.Tensor([1.2   5.    3.142], shape=(3,), dtype=float32)


### 13.02.03. Manipulating the data type and shape of a tensor

- `tf.cast()`: change data type

In [6]:
t_a_new = tf.cast(t_a, tf.int64)
print(t_a_new.dtype)

<dtype: 'int64'>


- rank: dim
- shape

useful fn

- `tf.transpose()`: change shape
- `tf.reshape()`: add new dim
- `tf.squeeze()`: squeeze an unnecessary dim

###### transposing a tensor

`tf.transpose(tensor)`

In [7]:
t = tf.random.uniform(shape=(3, 5))
t_tr = tf.transpose(t)
print(t.shape, '-->', t_tr.shape)

(3, 5) --> (5, 3)


###### reshaping a tensor

`tf.reshape(tensor, shape)`

In [8]:
t = tf.zeros((30,))
t_reshape = tf.reshape(t, shape=(5, 6))
print(t.shape, '-->', t_reshape.shape)

(30,) --> (5, 6)


###### removing the unnecessary dims

`tf.squeeze(tensor, axis)`

squeeze listed axis (#0, #1, ...)

In [9]:
t = tf.zeros((1, 2, 1, 4, 1))
t_sqz = tf.squeeze(t, axis=(2,4))
print(t.shape, '-->', t_sqz.shape)

(1, 2, 1, 4, 1) --> (1, 2, 4)


### 13.02.04. Applying mathematical operations to tensors 

- element-wise product
- matrix multiplication
- norm

standard norm dist

In [12]:
tf.random.set_seed(1)
t1 = tf.random.uniform(shape=(5,2), minval=-1.0, maxval=1.0)
t2 = tf.random.normal(shape=(5,2), mean=0.0, stddev=1.0)

`tf.multiply(tensor1, tensor2)`:  element-wise multiplication

In [13]:
t3 = tf.multiply(t1, t2).numpy()
print(t3)

[[-0.27  -0.874]
 [-0.017 -0.175]
 [-0.296 -0.139]
 [-0.727  0.135]
 [-0.401  0.004]]


`tf.math.reduce_mean()`: mean

`tf.math.reduce_sum()`: sum

`tf.math.reduce_std()`: standard deviation

In [14]:
t4 = tf.math.reduce_mean(t1, axis=0)
print(t4)

tf.Tensor([0.09  0.207], shape=(2,), dtype=float32)


$ t_1 \times t_2^T $ 

$ t_{5 \times 2} \times t_{2 \times 5} = t_{5 \times 5} $

`tf.linalg.matmul(tensor_a, tensor_b, transpose_a=False, transpose_b=False)`: matrix-matrix product

In [15]:
t5 = tf.linalg.matmul(t1, t2, transpose_b=True)
print(t5.numpy())

[[-1.144  1.115 -0.87  -0.321  0.856]
 [ 0.248 -0.191  0.25  -0.064 -0.331]
 [-0.478  0.407 -0.436  0.022  0.527]
 [ 0.525 -0.234  0.741 -0.593 -1.194]
 [-0.099  0.26   0.125 -0.462 -0.396]]


$ t_1^T \times t_2 $

$ t_{2 \times 5} \times t_{5 \times 2} = t_{2 \times 2} $

In [16]:
t6 = tf.linalg.matmul(t1, t2, transpose_a=True)
print(t6.numpy())

[[-1.711  0.302]
 [ 0.371 -1.049]]


`tf.norm(tensor, ord='', axis=None)`

ord: 
`'fro'`, `'euclidean'`, `1`, `2`, `np.inf`

$L^p$ e.g., $L^2$

In [17]:
norm_t1 = tf.norm(t1, ord=2, axis=1).numpy()
print(norm_t1)

[1.046 0.293 0.504 0.96  0.383]


### 13.02.05. Split, stack, and concatenate tensors

spliting, stacking 

`tf.split(tensor, num_or_size_splits=number)`

providing the num of splits

In [2]:
tf.random.set_seed(1)
t = tf.random.uniform((6,))
print(t.numpy())

t_splits = tf.split(t, num_or_size_splits=3)
[item.numpy() for item in t_spllits]

[0.16513085 0.9014813  0.6309742  0.4345461  0.29193902 0.64250207]


[array([0.16513085, 0.9014813 ], dtype=float32),
 array([0.6309742, 0.4345461], dtype=float32),
 array([0.29193902, 0.64250207], dtype=float32)]

providing the sizes of different splits

`tf.split(tensor, num_or_size_splits=[number, number])`

In [3]:
tf.random.set_seed(1)
t = tf.random.uniform((5,))
print(t.numpy())

t_splits = tf.split(t, num_or_size_splits=[3, 2])
[item.numpy() for item in t_splits]

[0.16513085 0.9014813  0.6309742  0.4345461  0.29193902]


[array([0.16513085, 0.9014813 , 0.6309742 ], dtype=float32),
 array([0.4345461 , 0.29193902], dtype=float32)]

`tf.concat([tensor, tensor], axis=number)`

In [4]:
A = tf.ones((3,))
B = tf.zeros((2,))
C = tf.concat([A, B], axis=0)
print(C.numpy())

[1. 1. 1. 0. 0.]


In [5]:
A = tf.ones((3,))
B = tf.zeros((3,))
S = tf.stack([A, B], axis=1)
print(S)

tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]], shape=(3, 2), dtype=float32)


[TensorFlow docs](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf)

### 13.02.06. Builing input pipleines using tf.data - the TensorFlow Dateaset API

`.fit()`

#### 13.02.06.01. Creating a TensorFlow Dataset from existing tensors

`tf.data.Dataset.from_tensor_slices()`
return `Dataset` (can iterate through the individual elements)

In [2]:
a = [1.2, 3.4, 7.5, 4.1, 5.0, 1.0]
ds = tf.data.Dataset.from_tensor_slices(a)
print(ds)

<TensorSliceDataset shapes: (), types: tf.float32>


In [3]:
for item in ds:
    print(item)

tf.Tensor(1.2, shape=(), dtype=float32)
tf.Tensor(3.4, shape=(), dtype=float32)
tf.Tensor(7.5, shape=(), dtype=float32)
tf.Tensor(4.1, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


create batches from dataset

In [4]:
ds_batch = ds.batch(3)
for i, elem in enumerate(ds_batch, 1):
    print('batch {}:'.format(i), elem.numpy())

batch 1: [1.2 3.4 7.5]
batch 2: [4.1 5.  1. ]


`.batch()` method has optional arg `drop_remainder`
`drop_remainder` useful for cases whten # of elements in teh tensor is NOT divisble by desired batch size

`shuffle`, `batch`, `repeat`

#### 13.02.06.02. Combining two tensors into a joint dataset

In [5]:
tf.random.set_seed(1)
t_x = tf.random.uniform([4,3], dtype=tf.float32)
t_y = tf.range(4)

In [6]:
ds_x = tf.data.Dataset.from_tensor_slices(t_x)
ds_y = tf.data.Dataset.from_tensor_slices(t_y)

ds_joint = tf.data.Dataset.zip((ds_x, ds_y))

for example in ds_joint:
    print('  x:', example[0].numpy(), 
          '  y:', example[1].numpy())

  x: [0.16513085 0.9014813  0.6309742 ]   y: 0
  x: [0.4345461  0.29193902 0.64250207]   y: 1
  x: [0.9757855  0.43509948 0.6601019 ]   y: 2
  x: [0.60489583 0.6366315  0.6144488 ]   y: 3


`zip()` fn joint dataset 

alternatively  
`tf.data.Dataset.from_tensor_slices()` 

In [7]:
ds_joint = tf.data.Dataset.from_tensor_slices((t_x, t_y))

for example in ds_joint:
    print('  x:', example[0].numpy(), 
          '  y:', example[1].numpy())

  x: [0.16513085 0.9014813  0.6309742 ]   y: 0
  x: [0.4345461  0.29193902 0.64250207]   y: 1
  x: [0.9757855  0.43509948 0.6601019 ]   y: 2
  x: [0.60489583 0.6366315  0.6144488 ]   y: 3


In [6]:
ds_trans = ds_joint.map(lambda x, y: (x*2-1.0, y))
for example in ds_trans:
    print('  x:', example[0].numpy(), 
          '  y:', example[1].numpy())

  x: [-0.6697383   0.80296254  0.26194835]   y: 0
  x: [-0.13090777 -0.41612196  0.28500414]   y: 1
  x: [ 0.951571   -0.12980103  0.32020378]   y: 2
  x: [0.20979166 0.27326298 0.22889757]   y: 3


`.map()` method

#### 13.02.06.03. Shuffle, batch, and repeat

In [7]:
tf.random.set_seed(1)
ds = ds_joint.shuffle(buffer_size=len(t_x))

for example in ds:
    print('  x:', example[0].numpy(), 
          '  y:', example[1].numpy())

  x: [0.9757855  0.43509948 0.6601019 ]   y: 2
  x: [0.4345461  0.29193902 0.64250207]   y: 1
  x: [0.16513085 0.9014813  0.6309742 ]   y: 0
  x: [0.60489583 0.6366315  0.6144488 ]   y: 3


`.shuffle` 
arg `buffer_size` 


In [8]:
ds = ds_joint.batch(batch_size=3, 
                    drop_remainder=False)
batch_x, batch_y = next(iter(ds))
print('Batch-x:\n', batch_x.numpy())
print('Batch-y:\n', batch_y.numpy())

Batch-x:
 [[0.16513085 0.9014813  0.6309742 ]
 [0.4345461  0.29193902 0.64250207]
 [0.9757855  0.43509948 0.6601019 ]]
Batch-y:
 [0 1 2]


In [12]:
ds = ds_joint.batch(3).repeat(count=2)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (3, 3) [0 1 2]
1 (1, 3) [3]
2 (3, 3) [0 1 2]
3 (1, 3) [3]


In [13]:
ds = ds_joint.repeat(count=2).batch(3)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (3, 3) [0 1 2]
1 (3, 3) [3 0 1]
2 (2, 3) [2 3]


In [14]:
## Order 1: shuffle -> batch -> repeat
tf.random.set_seed(1)
ds = ds_joint.shuffle(4).batch(2).repeat(3)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (2, 3) [2 1]
1 (2, 3) [0 3]
2 (2, 3) [0 3]
3 (2, 3) [1 2]
4 (2, 3) [3 0]
5 (2, 3) [1 2]


In [15]:
## Order 2: batch -> shuffle -> repeat
tf.random.set_seed(1)
ds = ds_joint.batch(2).shuffle(4).repeat(3)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (2, 3) [0 1]
1 (2, 3) [2 3]
2 (2, 3) [0 1]
3 (2, 3) [2 3]
4 (2, 3) [2 3]
5 (2, 3) [0 1]


- 13.01. TensorFlow and training performance
  - 13.01.01. Performance challenges
  - 13.01.02. What is TensorFlow?
  - 13.01.03. How we will learn TensorFlow
- 13.02. First steps with TensorFlow
  - 13.02.01. Installing TensorFlow
  - 13.02.02. Creating tensors in TensorFlow
  - 13.02.03. Manipulating the data type and shape of a tensor
  - 13.02.04. Applying mathematical operations to tensors
  - 13.02.05. Split, stack, and concatenate tensors
  - 13.02.06. Building input pipelines using tf.data – the TensorFlow Dataset API
    - 13.02.06.01. Creating a TensorFlow Dataset from existing tensors
    - 13.02.06.02. Combining two tensors into a joint dataset
    - 13.02.06.03. Shuffle, batch, and repeat
    - 13.02.06.04. Creating a dataset from files on your local storage disk
    - 13.02.06.05. Fetching available datasets from the `tensorflow_datasets` library
- 13.03. Building an NN model in TensorFlow
  - 13.03.01. The TensorFlow Keras API (tf.keras)
  - 13.03.02. Building a linear regression model
  - 13.03.03. Model training via the `.compile()` and `.fit()` methods 
  - 13.03.04. Building a multilayer perceptron for classifying flowers in the Iris dataset
  - 13.03.05. Evaluating the trained model on the test dataset
  - 13.03.06. Saving and reloading the trained model
- 13.04. Choosing activation functions for multilayer NNs
  - 13.04.01. Logistic function recap
  - 13.04.02. Estimating class probabilities in multiclass classification via the softmax function
  - 13.04.03. Broadening the output spectrum using a hyperbolic tangent
  - 13.04.04. Rectified linear unit activation
- 13.05. Summary